In [1]:
# %env BASE_DIR=/home/jupyter/snippets

In [2]:
# !gsutil cp gs://cs327e-open-access-2/shopify.zip .

In [3]:
# !unzip $BASE_DIR/shopify.zip

1. Create the Firestore collections and subcollections based on your design and populate them with the Shopify records.

In [4]:
import pandas as pd
from google.cloud import firestore
db = firestore.Client()
batch = db.batch()

df = pd.read_csv('shopify/categories.csv', sep=',', header=0, lineterminator='\n')
cat_rows = df.values.tolist()

df = pd.read_csv('shopify/apps.csv', sep=',', header=0, lineterminator='\n')
app_rows = df.values.tolist()

df = pd.read_csv('shopify/apps_categories.csv', sep=',', header=0, lineterminator='\n')
ac_rows = df.values.tolist()

# putting category documents into categories collection
# putting app documents into apps subcollection of each category document

i = 0

# categories collection
for cat_row in cat_rows:
    cat_record = {}
    cat_record['cat_id'] = cat_row[0]
    cat_record['title'] = cat_row[1]
    
    cat_ref = db.collection('categories').document(cat_row[0])
    
    batch.set(cat_ref, cat_record)
    i += 1
    if i > 400:
        i = 0
        batch.commit()
    
    # apps subcollection:
    for ac_row in ac_rows:
        if cat_row[0] == ac_row[1]:
            for app_row in app_rows:
                if app_row[0] == ac_row[0]:
                    app_record = {}
                    app_record['app_id'] = app_row[0]
                    app_record['url'] = app_row[1]
                    app_record['title'] = app_row[2]
                    app_record['developer'] = app_row[3]
                    app_record['developer_link'] = app_row[4]
                    app_record['icon'] = app_row[5]
                    app_record['rating'] = app_row[6]
                    app_record['review_count'] = app_row[7]
                    
                    app_ref = cat_ref.collection('apps').document(app_row[0])
                    
                    batch.set(app_ref, app_record)
                    i += 1
                    if i > 400:
                        i = 0
                        batch.commit()
                    
    batch.commit()

In [5]:
import pandas as pd
from google.cloud import firestore
db = firestore.Client()
batch = db.batch()

df = pd.read_csv('shopify/pricing_plans.csv', sep=',', header=0, lineterminator='\n')
pp_rows = df.values.tolist()

df = pd.read_csv('shopify/pricing_plan_features.csv', sep=',', header=0, lineterminator='\n')
ppf_rows = df.values.tolist()

df = pd.read_csv('shopify/key_benefits.csv', sep=',', header=0, lineterminator='\n')
kb_rows = df.values.tolist()

# putting app documents into apps collection
# putting pricing plane documents into pricing plans subcollection in each app document
# putting key benefits documents into key benefits subcollection in each app document

i = 0

# apps collection:
for app_row in app_rows:
    app_record = {}
    app_record['app_id'] = app_row[0]
    app_record['url'] = app_row[1]
    app_record['title'] = app_row[2]
    app_record['developer'] = app_row[3]
    app_record['developer_link'] = app_row[4]
    app_record['icon'] = app_row[5]
    app_record['rating'] = app_row[6]
    app_record['review_count'] = app_row[7]
    
    app_ref = db.collection('apps').document(app_row[0])
    
    batch.set(app_ref, app_record)
    i += 1
    if i > 400:
        i = 0
        batch.commit()
    
    # pricing_plans subcollection:
    for pp_row in pp_rows:
        if app_row[0] == pp_row[1]:
            pp_record = {}
            pp_record['plan_id'] = pp_row[0]
            pp_record['title'] = pp_row[2]
            pp_record['price'] = pp_row[3]
            features = []
            for ppf_row in ppf_rows:
                if pp_row[1] == ppf_row[1] and pp_row[0] == ppf_row[0]:
                    features.append(ppf_row[2])
            pp_record['features'] = features
            
            pp_ref = app_ref.collection('pricing_plans').document(pp_row[0])
            
            batch.set(pp_ref, pp_record)
            i += 1
            if i > 400:
                i = 0
                batch.commit()
                
    # key_benefits subcollection:
    for kb_row in kb_rows:
        if kb_row[0] == app_row[0]:
            kb_record = {}
            kb_record['title'] = kb_row[1]
            kb_record['description'] = kb_row[2]
                        
            kb_ref = app_ref.collection('key_benefits').document(kb_row[1].replace("/","_"))
            
            batch.set(kb_ref, kb_record)
            i += 1
            if i > 400:
                i = 0
                batch.commit()
                 
    batch.commit()

2. Get a count of the number of documents in each collection and subcollection.

In [6]:
print("Documents in categories collection:")
cats_ref = db.collection('categories')
result = cats_ref.count().get()[0][0].value
print(result)

Documents in categories collection:
12


In [7]:
print("Documents in apps subcollection in categories collection:")
for cat_ref in cats_ref.list_documents():
    result = cat_ref.collection('apps').count().get()[0][0].value
    print(cat_ref.id, ":", result)

Documents in apps subcollection in categories collection:
26a72de0d02e0e4e5f615332d61a878e : 133
30a930262efca129caafcd586bc7e6fe : 83
30ea2315e910c5e5335de42d5e87dca5 : 602
5eb4e29e50e3f178acc614236ed107f4 : 957
64430ad2835be8ad60c59e7d44e4b0b1 : 310
737ad50051083aa051d127a53b3ac0da : 190
9fe78acd078fc030de72c896adc93a2d : 251
bd125b49ce6b0425d29e33cb8b49a496 : 176
c3f9db73c29bc33607778f9935c3c4dc : 1537
c576a841fd4f333a6f074d68e76a1d37 : 249
c769c2bd15500dd906102d9be97fdceb : 794
e44cd5379a3166568741a07cc81ef6e5 : 101


In [8]:
print("Documents in apps collection:")
apps_ref = db.collection('apps')
result = apps_ref.count().get()[0][0].value
print(result)

Documents in apps collection:
3547


In [9]:
print("Documents in pricing plans and key benefits subcollections in apps collection:")
i = 0
for app_ref in apps_ref.list_documents():
    PPresult = app_ref.collection('pricing_plans').count().get()[0][0].value
    KBresult = app_ref.collection('key_benefits').count().get()[0][0].value
    print(app_ref.id, ":", sep="")
    print('    pricing_plans:', PPresult)
    print('    key_benefits:', KBresult)
    i += 1
    # just so that this doesn't take up the entire page
    if i > 10:
        break

Documents in pricing plans and key benefits subcollections in apps collection:
00289a9f-9f12-45b1-963b-67e78403f7c7:
    pricing_plans: 1
    key_benefits: 3
00312cea-3595-4d65-a60d-6e8e5b7d6897:
    pricing_plans: 2
    key_benefits: 3
003b3dbd-1295-48cb-a3e8-8f3a0e8fc6df:
    pricing_plans: 3
    key_benefits: 3
004cc187-6def-4058-a7f3-78e6edfe2e3c:
    pricing_plans: 1
    key_benefits: 3
005d6c7a-9eb9-4b35-bdd4-040d383e86cf:
    pricing_plans: 1
    key_benefits: 3
0067a20e-e307-490e-acc5-8442d4bdca4b:
    pricing_plans: 1
    key_benefits: 3
008ce864-6073-4b61-b133-265ea6afa893:
    pricing_plans: 1
    key_benefits: 3
00951791-6206-4780-ba17-5e57520c277c:
    pricing_plans: 1
    key_benefits: 3
00c5d020-0877-4534-88b8-f05fd4cf5fa6:
    pricing_plans: 4
    key_benefits: 3
00d5c1eb-5b16-439b-8b84-c7abe921e6e4:
    pricing_plans: 3
    key_benefits: 3
00dc6a76-76ba-42f3-92d8-1103331cd79c:
    pricing_plans: 1
    key_benefits: 3


3. List the top 10 "Productivity" apps (whose category.title = "Productivity") sorted by their rating in descending order. Return the id, title, developer, rating and reviews_count for those apps. Limit the results to the first 10 records. Note: this query refers to access pattern #1.

In [10]:
# figuring out the category ID of the Productivity category by querying the category collection and turning the result into a dictionary
prod_cid = cats_ref.where('title', '==', 'Productivity').get()[0].to_dict()['cat_id']
prapps_ref = cats_ref.document(prod_cid).collection('apps')
attr = ['app_id', 'title', 'developer', 'rating', 'review_count']
query = prapps_ref.select(attr).order_by('rating', direction=firestore.Query.DESCENDING).limit(10)
results = query.stream()
for result in results:
    rd = result.to_dict()
    for attribute in attr:
        print(attribute, ': ', rd[attribute], ', ', sep = "", end = "")
    print()    

app_id: fe5ae45c-379c-42bf-be7d-b5f6e15fc13c, title: Customer Tags, developer: Union Works Apps, rating: 5.0, review_count: 21, 
app_id: f99bb1e3-f326-4f10-8901-491652e9809b, title: Order Tagger, developer: Union Works Apps, rating: 5.0, review_count: 68, 
app_id: f864e3bd-da0e-41dc-be65-984325331475, title: SilkRoad ‑ Facebook Auto Ads, developer: SilkRoad, rating: 5.0, review_count: 2, 
app_id: f5344e64-9cda-4d97-b198-2aaeb5170518, title: Xporter Data Export Tool, developer: Modd Apps Inc., rating: 5.0, review_count: 223, 
app_id: ed77a32d-0fa3-458b-b639-e01ea7b78ec0, title: FraudBlock Fraud Prevention, developer: ShopFox, rating: 5.0, review_count: 6, 
app_id: e0e231d6-4988-4a8c-ad84-cf18d0f38738, title: Automation Fox, developer: Automation Fox, rating: 5.0, review_count: 1, 
app_id: d75fa395-3a4d-41c3-88d1-2f802c1e2411, title: AirPower, developer: BaseGenius, rating: 5.0, review_count: 1, 
app_id: d0be03ff-f74d-4fb5-8d2c-79541a1aea1b, title: Excelify, developer: Excelify.io, ratin

4. List the top 10 apps with the highest number of reviews (based on apps.review_count). Return the id, title, developer, rating and reviews_count for those apps. Order the results by reviews_count in descending order. Note: this query refers to access pattern #2.

In [11]:
query = apps_ref.select(attr).order_by('review_count', direction=firestore.Query.DESCENDING).limit(10)
results = query.stream()
for result in results:
    rd = result.to_dict()
    for attribute in attr:
        print(attribute, ': ', rd[attribute], ', ', sep = "", end = "")
    print()    

app_id: d9f142ee-b141-4dc4-9353-173db61d2eb0, title: Privy ‑ Exit Pop Ups & Email, developer: Privy, rating: 4.7, review_count: 23078, 
app_id: 78ea0810-c008-4a4e-a82f-de0c790e3286, title: Free Shipping Bar, developer: Hextom, rating: 4.9, review_count: 8737, 
app_id: b88488b0-9912-44d3-b736-224c36f09d95, title: Sales Pop ‑ Popup Notification, developer: CartKit, rating: 4.8, review_count: 6905, 
app_id: e528a60e-94f8-4e92-80e2-5bc6013b8283, title: BEST Currency Converter, developer: Grizzly Apps, rating: 4.8, review_count: 5986, 
app_id: be2640c4-01b5-4d52-9f68-cae8c0734d0d, title: Recart FB Messenger Marketing, developer: Recart, rating: 4.8, review_count: 5596, 
app_id: 70bff9e0-4316-4cc6-84ce-92fcd1bc6925, title: EU Cookie Bar ‑ Cookie GDPR, developer: Booster Apps, rating: 4.7, review_count: 5259, 
app_id: 171816e2-27d4-4552-a65e-ab44a312fe04, title: Sales Pop Master ‑ Countdown, developer: Autoketing, rating: 4.8, review_count: 4931, 
app_id: 9025eff0-d714-4df1-930f-43f5582979ad,